In [10]:
%reload_ext autoreload
%autoreload 2

In [11]:
#export
import numpy as np
from rdkit import Chem

from rdkit.Chem.AllChem import GetMACCSKeysFingerprint
from descriptastorus.descriptors import DescriptorGenerator, MorganCounts, Morgan, RDKit2D, RDKitFPBits, FeatureMorganCounts, FeatureMorgan, AtomPair, AtomPairCounts

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('/home/marcossantana/Documentos/GitHub/ADMETBox/data/PGP/Metrabase/trainset.csv')

In [6]:
smiles = data.processed_smiles.tolist()

In [7]:
#export
class MACCSGenerator(DescriptorGenerator):
    """Computes MACCS bitvector"""
    NAME = "MACCS"
    def __init__(self):
            
        DescriptorGenerator.__init__(self)
        # specify names and numpy types for all columns
        maccs = [("maccs-%d"%d, np.uint8) for d in range(166)]
        self.columns += maccs

    def calculateMol(self, m, smiles, internalParsing=False):
        counts = list(GetMACCSKeysFingerprint(m))[1: ]
        return counts       
    

class Fingerprinter():
    def __init__(self, smiles):
        self.smiles = smiles
        
        
    def _define_generators(self, gen, **kwargs):
    
        '''Define feature generator.

        Arguments:
        gen : str
            Name of the generator to use. Options: 

            ecfp : Morgan fingerprints (extended connectivity fingerprint)
            fcfp : Feature-based Morgan fingerprints (functional connectivity fingerprint)
            atom_pairs : Atom pairs as defined in https://pubs.acs.org/doi/10.1021/ci00046a002

        nbits : int
            Final number of bits in the fingerprint

        radius : int
            Radius around central atom to calculate Morgan fingerprints'''
    
        radius = kwargs.get('radius', 3)
        nbits = kwargs.get('nbits', 2048)


        if gen == 'ecfp':
            feature_generator = Morgan(radius=radius,nbits=nbits)
            
        elif gen == 'atom_pairs':
            feature_generator = AtomPairCounts(nbits=nbits)
            
        elif gen == 'fcfp':
            feature_generator = FeatureMorgan(radius=radius,nbits=nbits)
            
        elif gen == 'maccs':
            feature_generator = MACCSGenerator()
            
        return feature_generator 


    def generate_fingerprint(self, gen, **kwargs):
        '''Calculate features for a list of SMILES using a feature generator.

        Arguments:
        gen : str
            Name of the generator to use. Options: 

            ecfp : Morgan fingerprints (extended connectivity fingerprint)
            fcfp : Feature-based Morgan fingerprints (functional connectivity fingerprint)
            atom_pairs : Atom pairs as defined in https://pubs.acs.org/doi/10.1021/ci00046a002


        nbits : int
            Final number of bits in the fingerprint

        radius : int
            Radius around central atom to calculate Morgan fingerprints'''
    
        if not isinstance(gen, str):
            raise TypeError('Please provide a string that represents a valid generator name.')

        if gen not in ['maccs','ecfp', 'fcfp', 'atom_pairs']:
            raise ValueError("Please provide a valid generator. Viable options are: 'maccs','ecfp', 'fcfp', 'atom_pairs'")

        
        feature_generator = self._define_generators(gen=gen, **kwargs)
        return np.array(feature_generator.processSmiles(self.smiles)[1])[:, 1:]
    
        
    def __len__(self): 
        return len(self.smiles)
    
    def __getitem__(self, i):
        return self.smiles[i]
    
    def __str__(self):
        return 'Size of SMILES dataset: {}\nFirst SMILES: {}'.format(len(self.smiles), self.smiles[0])

In [8]:
fp_class = Fingerprinter(smiles)

In [9]:
print(fp_class)

Size of SMILES dataset: 766
First SMILES: CN(C)CCOc1ccc(/C(=C(/CCCl)c2ccccc2)c2ccccc2)cc1


In [10]:
X = fp_class.generate_fingerprint(gen='atom_pairs')

In [11]:
X.shape

(766, 2048)

# Testing

In [ ]:
import unittest

In [12]:
class TestFingerprinter(unittest.TestCase):
    
    def test_fingerprint_shape(self):
        smiles = ['CN(C)CCOc1ccc(/C(=C(/CCCl)c2ccccc2)c2ccccc2)cc1']
        fp_class = Fingerprinter(smiles)
        sizes = [166, 1024, 1024, 1024]
        for fp_type, size in list(zip(['maccs','ecfp','fcfp','atom_pairs'], sizes)):
            print(fp_type, size)
            x = fp_class.generate_fingerprint(gen=fp_type, nbits=size)

            self.assertEqual(x.shape, (len(smiles), size))

In [13]:
TestFingerprinter().test_fingerprint_shape()

maccs 166
ecfp 1024
fcfp 1024
atom_pairs 1024


In [12]:
from utils import Fingerprinter

In [13]:
smiles = ['c1ccccc1']

In [14]:
gen_fps = Fingerprinter.Fingerprinter(smiles)

In [16]:
gen_fps.generate_fingerprint(gen='maccs').shape

(1, 166)

In [17]:
print(gen_fps)

Size of SMILES dataset: 1
First SMILES: c1ccccc1
